# k-Nearest-Neighbors from Scratch

Let's look at coding up one of the simplest machine learning (ML) algorithms: k-Nearest-Neighbors (kNN). kNN is an ML algorithm that classifies an object based on some distance metric from other labeled objects. 

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [2]:
# first we need a function to compute the distance metric. We can use several different metrics but let's
# just start with the standard euclidian distance. 
euc_dist = lambda x: np.linalg.norm(x[1]-x[0])

In [172]:
# great! now let's build a classifier
def test_set(train, test, k=5, distance_metric = euc_dist, reverse = False):
    correct = 0
    for index, row in test.iterrows():
        prediction = predict(train,row[:-1], k, distance_metric, reverse)
        if (prediction[-1] == row[-1]):
            correct += 1
    print("Congrats, you have {:.2%} accuracy!\n\n".format(correct/len(test)))
    
    return correct/len(test)

def predict(train, test_vec, k=5, distance_metric = euc_dist, reverse = False):
    distance = []

    # compute distance metric for every instance of the training set
    for index, row in train.iterrows():
        dist = distance_metric([row[:-1],test_vec])
        distance.append([row,dist])
    distance.sort(key = lambda x: x[1], reverse=reverse)

    # now look at the k-nearest-neighbors and have them vote on the label!
    pred = vote(distance, k)
    
    return (test_vec,pred[0])


def vote(distance, k):
    knn = distance[:k]
    occurances = Counter(row[0]['class'] for row in knn)
    occurances = [(key, value) for key, value in sorted(occurances.items(), key=lambda item: item[1], reverse = True)]
    
    # if we have a tie we want to reduce k until we break the tie!
    if ((len(occurances) > 1) and (occurances[0][1] == occurances[1][1])):
        print('tie!\n')
        answer = vote(distance, k-1)
    else:
        answer = occurances[0]
    
    return answer

In [156]:
# here I will be using sklearn but only to grab the data and split it into a train and test set. 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris_data = load_iris()
Y_iris = iris_data.target
iris_data = pd.DataFrame(iris_data.data,
                            columns = iris_data.feature_names)
iris_data = pd.concat([iris_data,pd.Series(Y_iris)],axis=1)
iris_data.rename(columns = {0:'class'},inplace=True)
train, test = train_test_split(iris_data, test_size = 0.2, shuffle = True)
train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
21,5.1,3.7,1.5,0.4,0
73,6.1,2.8,4.7,1.2,1
135,7.7,3.0,6.1,2.3,2
128,6.4,2.8,5.6,2.1,2
53,5.5,2.3,4.0,1.3,1


In [157]:
# now let's test different values of k for some parameter tuning
k_list = [3,5,7,9,11,13,15] # this way I'll get no 3-way ties

for k in k_list:
    print('Testing k = {}...\n'.format(k))
    test_set(train, test, k)

Testing k = 3...

Congrats, you have 96.67% accuracy!


Testing k = 5...

Congrats, you have 96.67% accuracy!


Testing k = 7...

Congrats, you have 96.67% accuracy!


Testing k = 9...

Congrats, you have 96.67% accuracy!


Testing k = 11...

Congrats, you have 96.67% accuracy!


Testing k = 13...

Congrats, you have 96.67% accuracy!


Testing k = 15...

Congrats, you have 96.67% accuracy!




In [163]:
# this is pretty good. If we were being good about ML we would perform some cross validation but we are really
# just demonstrating the method here. I would like to see how these results differ with different metrics!
# Let's test out Cosine distance ... Note there are MANY distance measures and diving into 
# which ones perform the best could consititute an entirely separate post.

cosine_distance = lambda x: (np.dot(x[0],x[1]))/((np.linalg.norm(x[0]))*(np.linalg.norm(x[1])))

In [177]:
test_set(train, test, k = 1, distance_metric = cosine_distance, reverse = True)

Congrats, you have 100.00% accuracy!




1.0

In [160]:
L1_distance = lambda vec: np.linalg.norm(vec[1]-vec[0],ord=1)
test_set(train, test, k = 3, distance_metric = L1_distance)

Congrats, you have 96.67% accuracy!




0.9666666666666667

1.0